
![title](Picture.jpg)
# Looming Stimulus Paradigm Analysis  




In [1]:
import numpy as np
import pandas as pd
import csv
import os, shutil
import glob
import easygui
import matplotlib.pyplot as plt
from IPython.display import Image
import math
from math import sqrt
import matplotlib.mlab as mlab
import statistics as st
from matplotlib.patches import Ellipse
import matplotlib.ticker as ticker
import seaborn as sns
from math import pi
from scipy import stats
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [2]:
skips=36
FPS=30
xposttime = 60
ans = easygui.boolbox(msg='Load a collection?', title='Collection', choices=('[Y]es', '[N]o'));
if ans == True:
    listcols=[];
    path=('tag_folder/collections'); #place where new files are
    allFiles = glob.glob(path + '/*.txt');

    for file_ in allFiles:
            base = os.path.basename(file_);
            base1 = os.path.splitext(base)[0];
            listcols.append(base1);

    choice = easygui.choicebox('Which collection would you like to load?', 'Collection', listcols);
    dates = pd.read_csv('tag_folder/collections/'+str(choice)+'.txt', index_col = False)
    dates = dates.date
    noe=len(dates)
    
elif ans ==False:
    listq0 =['Number of experiments']
    no = easygui.multenterbox("Enter number of experiments", "Number of experiments", listq0)
    noe = int(no[0])
    e=1
    exp_list=[]
    for i in range(noe):
        exp= 'Experiment '+str(e)
        exp_list.append(exp)
        e=e+1

    dates = []
    dates = easygui.multenterbox('Enter experiments to be analysed', 'Experiment', exp_list)
    
    ans2 = easygui.boolbox(msg='Save as collection?', title='Collection', choices=('[Y]es', '[N]o'), image=None, default_choice='Yes', cancel_choice='No');
    if ans2 == True:
        listq1 =['Collection name']
        cn=[]
        cn = easygui.multenterbox("Enter name of collection", "Name collection", listq1)
        cn=str(cn[0])
        cntable = pd.DataFrame({'date':dates})
        cntable.to_csv('tag_folder/collections/'+str(cn)+'.txt')
    elif ans2 == False:
        pass
    
answer=easygui.ynbox('Normal experiment details?', 'Experiment details');
listq =['Time the first dot starts','Total length of the trial','speed of dot (3.85)','Time the trial started']
experiment_details = []
if answer == False:
    experiment_details = easygui.multenterbox('Enter the experiment details', 'Experiment details', listq)
    Time_dots_start = int(experiment_details[0])
    length_of_trial = int(experiment_details[1])
    speed_of_dot = int(experiment_details[2])
    time_start = int(experiment_details[3])
else:
    Time_dots_start = 33.1
    length_of_trial = 150
    speed_of_dot = 4.85
    time_start = 0
    
header = ['Trial_time','Recording_time','X_center','Y_center','Area','Areachange','Elongation','Distance_moved','Heading_to_point','Mobility_continuous','Result_1','Result_2']
listall=[]
allFiles=[]

listallconditions=[]
listalltreatment=[]
listallsubject=[]
listallstrain=[]

for e in range(noe):
    list_=[]
    path=('Temp_bin/' + dates[e]) #place where new files are
    allFiles = glob.glob(path + '/*.txt')
    if len(allFiles)>0:
        frame = pd.DataFrame()
        for file_ in allFiles:
            df = pd.read_csv(file_, names=header,index_col=False, encoding ='utf-16',  skiprows=skips)
            for dfs in df:
                try: 
                    df=df[df.Area != '-']
                except: 
                    pass
                base = os.path.basename(file_)
                base1 = os.path.splitext(base)[0]
                df['Trial'] =  base1
            list_.append(df)
    else:
        easygui.msgbox('Reload data files into temp_bin folder and try again')

    listdf=[]   
    for df1 in list_:
        df1 = df1.filter(['Trial_time','Recording_time','X_center','Y_center'], axis=1) #filters columns
        df1 = df1.reset_index(drop=True) #resets index column to match new row count
        df1 = df1.astype(float) #float it
        listdf.append(df1)

    length,trial_list,trial = int((len(listdf))),[],100
    for i in range(length):
        trial=trial+1
        trial_list.append(trial)
    
    listcondition=[]
    listtreatment=[]
    listsubject=[]
    liststrain=[]
    
    if os.path.exists('tag_folder/'+str(dates[e])+'.csv'):
        tag = pd.read_csv('tag_folder/'+str(dates[e])+'.csv', index_col = False)
        condition = []
        condition = tag.Condition
        treatment = []
        treatment = tag.Treatment
        subject = []
        subject = tag.Subject
        strain = []
        strain = tag.Strain
        
        listcondition.append(condition)
        listtreatment.append(treatment)
        listsubject.append(subject)
        liststrain.append(strain)
        
    else:
        condition = []
        condition = easygui.multenterbox('Enter condition of each trial', 'Condition', trial_list)
        treatment = []
        treatment = easygui.multenterbox('Enter treatment of each trial', 'Treatment', trial_list)
        subject = []
        subject = easygui.multenterbox('Enter subject of each trial', 'Subject', trial_list)
        strain = []
        strain = easygui.multenterbox('Enter Strain of each trial', 'Strain', trial_list)    

        tags = pd.DataFrame({'Trial':trial_list,'Condition':condition,'Treatment':treatment,'Subject':subject,'Strain':strain}) 
        tags['Date']=dates[0]
        tags.to_csv('tag_folder/'+str(dates[e])+'.csv')
    
    listdf1,q=[],0
    for df1 in listdf:  
        df1['date']= dates[e]
        df1['trial']=trial_list[q]
        df1['condition']=condition[q]
        df1['treatment']=treatment[q]
        df1['subject']=subject[q]
        df1['strain']=strain[q]
        df1['X_center'] = df1.X_center
        df1['Y_center'] = df1.Y_center
        df1['Recording_time'] = df1.Recording_time

        df1 = df1.filter(['date','trial','condition','treatment','subject','strain','Recording_time','X_center','Y_center'], axis=1)
        
        listallconditions.append(condition[q])
        listalltreatment.append(treatment[q])
        listallsubject.append(subject[q])
        listallstrain.append(strain[q])
        listdf1.append(df1)
        listall.append(df1)
        q=q+1
    
listdf1=listall

for df1 in listdf1:
    distance_moved=[] 
    for i in range(1,len(df1.X_center)):
        a = df1.X_center[i]- df1.X_center[i-1] #a = current x coordinate - previous x coordinate
        b = df1.Y_center[i]- df1.Y_center[i-1] #b = current y coordinate - previous y coordinate
        truedistance = sqrt(a**2 + b**2) # hypotenuse calculation
        distance_moved.append(truedistance)
    distance_moved.insert(0,0)
    df1['Distance_moved']=distance_moved # replace Distance_moved column with new true distance"

for df1 in listdf1: 
    listx, listy, listdm = [],[],[]
    for i in range(1, len(df1.Distance_moved)):
        if df1.Distance_moved[i] > 2.5:
            x = ((df1.X_center[i-1] + df1.X_center[i+1])/2)
            y = ((df1.Y_center[i-1] + df1.Y_center[i+1])/2)
            dm = sqrt((x - df1.X_center[i-1])**2 + (y - df1.Y_center[i-1])**2)
        else:
            x = df1.X_center[i]
            y = df1.Y_center[i]
            dm = df1.Distance_moved[i]
            
        listx.append(x)
        listy.append(y)  
        listdm.append(dm)
    listdm.insert(0,0) 
    listx.insert(0,0)               
    listy.insert(0,0) 
    
    df1['Distance_moved']=listdm
    df1['X_center']=listx
    df1['Y_center']=listy

# Tag phases

In [3]:
for df1 in listdf1:

    stims = int(df1.treatment[0]) ## number of stimulus
    
    if stims == 0:
        stims = 6
    
    liststim=[]
    for stim in range(stims):
        stimulus = Time_dots_start+((stim+1)*speed_of_dot)
        liststim.append(stimulus)

    maxstim = max(liststim)

    liststimulus=[]
    for i in range(0, len(df1.Recording_time)): 
        if df1.Recording_time[i] < (liststim[0]-speed_of_dot):
            stimulus = 'pre'
            liststimulus.append(stimulus)
        else:
            pass
     
    s=0 
    for p in liststim:
        for i in range(0, len(df1.Recording_time)):
            if s == 0:
                if df1.Recording_time[i] > (liststim[s]-speed_of_dot) and df1.Recording_time[i] <=liststim[s]:
                    stimulus = 'stimulus_'+str(s+101)
                    liststimulus.append(stimulus) 
                else:
                    pass
            else:
                if df1.Recording_time[i] > (liststim[s-1]) and df1.Recording_time[i] <=liststim[s]:
                    stimulus = 'stimulus_'+str(s+101)
                    liststimulus.append(stimulus) 
                else:
                    pass
     
        s=s+1
        
    for i in range(0, len(df1.Recording_time)):
        if df1.Recording_time[i] >= maxstim:
            if df1.Recording_time[i] <= (maxstim+xposttime):
                stimulus = 'post'
            else:
                stimulus = 'xpost'
            liststimulus.append(stimulus)
        else:
            pass
        
    if len(liststimulus) > len(df1):
        liststimulus.pop()
    
    if len(liststimulus) < len(df1):
        liststimulus.insert(0,'pre')
    

    df1['phase'] = liststimulus

KeyboardInterrupt: 

# Freezing

In [ ]:
FT = 0.1 #Define freezing threshold. Default = 0.1
for df1 in listdf1: # iterate through trials
    listfreeze=[]
    for i in range(5, len(df1.Recording_time)): #iterate through each frame
        try: #attempt to sum up -5 and + 5 frame from focal frame
            if (df1.Distance_moved[i]+df1.Distance_moved[i-1]+df1.Distance_moved[i-2]+df1.Distance_moved[i-3]+df1.Distance_moved[i-4]+df1.Distance_moved[i-5]+df1.Distance_moved[i+1]+df1.Distance_moved[i+2]+df1.Distance_moved[i+3]+df1.Distance_moved[i+4]+df1.Distance_moved[i+5]) < FT:
                freeze=1 # if sum < FT then we tag as a freezing frame
            else:
                freeze=0 # if sum > FT then we tag as a freezing frame
        except:
            freeze=0 # if for some reason there is an error: return 0
        listfreeze.append(freeze) # append tag data into a list
        
    for q in range(5): 
        listfreeze.insert(0,0) # fill in the first 5 frame with 0 as it is not possible to calculate for them
    df1['freezing']=listfreeze # make freeze tag column
    
#Summarizing freezing behaviour
listfreezepost,listfreezepre, listtotalfreeze = [],[],[]
listfreeze1,listfreeze2,listfreeze3,listfreeze4,listfreeze5,listfreeze6,listfreeze7,listfreeze8,listfreeze9,listfreeze10,listfreeze11,listfreeze12,listfreeze13,listfreeze14,listfreeze15,listfreeze16,listfreeze17,listfreeze18=[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
listoflists=[listfreeze1,listfreeze2,listfreeze3,listfreeze4,listfreeze5,listfreeze6,listfreeze7,listfreeze8,listfreeze9,listfreeze10,listfreeze11,listfreeze12,listfreeze13,listfreeze14,listfreeze15,listfreeze16,listfreeze17,listfreeze18]

for df1 in listdf1:
    summ = df1.groupby('phase')['freezing'].sum()  # Group freezing data by phase
    listfreezepost.append((100/xposttime)*(summ[0]/FPS)) # percent of time freezing post stress
    listfreezepre.append((100/Time_dots_start)*(summ[1]/FPS)) # percent of time freezing pre stress
    
    for i in range (0,len(listoflists)): #iterate through number of dot phases
        try:
            listoflists[i].append((speed_of_dot/xposttime)*(summ[i+1]/FPS)) #percent of time freezing per phase
        except:
            listoflists[i].append(0) # if none or 0, return 0
            
    listtotalfreeze.append((100/length_of_trial)*(df1['freezing'].sum())/FPS) #percent of trial freezing

#Make a table with all freezing data
freezetable = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'Pre_stress':listfreezepre,'Post_stress':listfreezepost,'S1':listfreeze1,'S2':listfreeze2,'S3':listfreeze3,'S4':listfreeze4,'S5':listfreeze5,'S6':listfreeze6,'S7':listfreeze7,'S8':listfreeze8,'S9':listfreeze9,'S10':listfreeze10,'S11':listfreeze11,'S12':listfreeze12,'S13':listfreeze13,'S14':listfreeze14,'S15':listfreeze15,'S16':listfreeze16,'S17':listfreeze17,'S18':listfreeze18, 'total':listtotalfreeze})
freezetable.to_csv('data/Freezing.csv') #export table

dfc = pd.concat(listdf1)
dfc.to_csv('data/all.csv');   

# Depth

In [ ]:
for df1 in listdf1:
    listy=[]
    for i in range(0, len(df1.Recording_time)):
        Y=df1.Y_center[i]+3.5
        listy.append(Y)
        
    df1['Depth']=listy

stress = speed_of_dot*FPS
prestress = Time_dots_start*FPS
poststress = (length_of_trial-(Time_dots_start+(stims*(stress/FPS))))*FPS

listdepthpost,listdepthpre=[],[]
listdepth1,listdepth2,listdepth3,listdepth4,listdepth5,listdepth6,listdepth7,listdepth8,listdepth9,listdepth10,listdepth11,listdepth12=[],[],[],[],[],[],[],[],[],[],[],[]
listdepth13,listdepth14,listdepth15,listdepth16,listdepth17,listdepth18 = [],[],[],[],[],[]
listtotaldepth=[]

for df1 in listdf1:
    summ=df1.groupby('phase')['Depth'].sum()
    
    listdepthpost.append((summ[0]/(xposttime*FPS)))
    listdepthpre.append((summ[1]/prestress))
    try:
        listdepth1.append((summ[2]/stress))
    except:
        listdepth1.append(0)
    try:
        listdepth2.append((summ[3]/stress))
    except:
        listdepth2.append(0)
    try:
        listdepth3.append((summ[4]/stress))
    except:
        listdepth3.append(0)
    try:
        listdepth4.append((summ[5]/stress))
    except:
        listdepth4.append(0)
    try:
        listdepth5.append((summ[6]/stress))
    except:
        listdepth5.append(0)
    try:
        listdepth6.append((summ[7]/stress))
    except:
        listdepth6.append(0)
    try:
        listdepth7.append((summ[8]/stress))
    except:
        listdepth7.append(0)
    try:
        listdepth8.append((summ[9]/stress))
    except:
        listdepth8.append(0)
    try:
        listdepth9.append((summ[10]/stress))
    except:
        listdepth9.append(0)
    try:
        listdepth10.append((summ[11]/stress))
    except:
        listdepth10.append(0)
    try:
        listdepth11.append((summ[12]/stress))
    except:
        listdepth11.append(0)
    try:
        listdepth12.append((summ[13]/stress))
    except:
        listdepth12.append(0)
    try:
        listdepth13.append((summ[14]/stress))
    except:
        listdepth13.append(0)
    try:
        listdepth14.append((summ[15]/stress))
    except:
        listdepth14.append(0)
    try:
        listdepth15.append((summ[16]/stress))
    except:
        listdepth15.append(0)
    try:
        listdepth16.append((summ[17]/stress))
    except:
        listdepth16.append(0)
    try:
        listdepth17.append((summ[18]/stress))
    except:
        listdepth17.append(0)
    try:
        listdepth18.append((summ[19]/stress))
    except:
        listdepth18.append(0)
        
    listtotaldepth.append(df1['Depth'].mean())

    
depthtable = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'Pre_stress':listdepthpre,'Post_stress':listdepthpost,'S1':listdepth1,'S2':listdepth2,'S3':listdepth3,'S4':listdepth4,'S5':listdepth5,'S6':listdepth6,'S7':listdepth7,'S8':listdepth8,'S9':listdepth9,'S10':listdepth10,'S11':listdepth11,'s12':listdepth12,'s13':listdepth13,'s14':listdepth14,'s15':listdepth15,'s16':listdepth16,'s17':listdepth17,'s18':listdepth18, 'total':listtotaldepth})
depthtable.to_csv('data/Depth.csv')

# Speed

In [ ]:
stress = speed_of_dot
prestress = Time_dots_start
poststress = length_of_trial-(Time_dots_start+(stims*stress))

listtotalspeed=[]
listspeedpost,listspeedpre=[],[]
listspeed1,listspeed2,listspeed3,listspeed4,listspeed5,listspeed6,listspeed7,listspeed8,listspeed9,listspeed10,listspeed11,listspeed12=[],[],[],[],[],[],[],[],[],[],[],[]
listspeed13,listspeed14,listspeed15,listspeed16,listspeed17,listspeed18 = [],[],[],[],[],[]
for df1 in listdf1:
    listmove=[]
    listphase=[]
    for i in range(0,len(df1.Recording_time)):
        if df1.freezing[i] ==0:
            move = df1.Distance_moved[i]
            phase = df1.phase[i]
            listmove.append(move)
            listphase.append(phase)
        else:
            pass
    
    moving = pd.DataFrame({'Distance_moved':listmove,'phase':listphase})

    summ=moving.groupby('phase')['Distance_moved'].sum()
    summ2=moving.groupby('phase')['Distance_moved'].count()
    
    listspeedpost.append((summ[0]/(summ2[0]/FPS)))
    listspeedpre.append((summ[1]/(summ2[1]/FPS)))
    
    try:
        listspeed1.append((summ[2]/(summ2[2]/FPS)))
    except:
        listspeed1.append(0)
    try:
        listspeed2.append((summ[3]/(summ2[3]/FPS)))
    except:
        listspeed2.append(0)
    try:
        listspeed3.append((summ[4]/(summ2[4]/FPS)))
    except:
        listspeed3.append(0)
    try:
        listspeed4.append((summ[5]/(summ2[5]/FPS)))
    except:
        listspeed4.append(0)
    try:
        listspeed5.append((summ[6]/(summ2[6]/FPS)))
    except:
        listspeed5.append(0)
    try:
        listspeed6.append((summ[7]/(summ2[7]/FPS)))
    except:
        listspeed6.append(0)
    try:
        listspeed7.append((summ[8]/(summ2[8]/FPS)))
    except:
        listspeed7.append(0)
    try:
        listspeed8.append((summ[9]/(summ2[9]/FPS)))
    except:
        listspeed8.append(0)
    try:
        listspeed9.append((summ[10]/(summ2[10]/FPS)))
    except:
        listspeed9.append(0)
    try:
        listspeed10.append((summ[11]/(summ2[11]/FPS)))
    except:
        listspeed10.append(0)
    try:
        listspeed11.append((summ[12]/(summ2[12]/FPS)))
    except:
        listspeed11.append(0)
    try:
        listspeed12.append((summ[13]/(summ2[13]/FPS)))
    except:
        listspeed12.append(0)
    try:
        listspeed13.append((summ[14]/(summ2[14]/FPS)))
    except:
        listspeed13.append(0)
    try:
        listspeed14.append((summ[15]/(summ2[15]/FPS)))
    except:
        listspeed14.append(0)
    try:
        listspeed15.append((summ[16]/(summ2[16]/FPS)))
    except:
        listspeed15.append(0)
    try:
        listspeed16.append((summ[17]/(summ2[17]/FPS)))
    except:
        listspeed16.append(0)
    try:
        listspeed17.append((summ[18]/(summ2[18]/FPS)))
    except:
        listspeed17.append(0)
    try:
        listspeed18.append((summ[19]/(summ2[19]/FPS)))
    except:
        listspeed18.append(0)
        
    listtotalspeed.append((df1['Distance_moved'].mean())*30)
    
speedtable = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'Pre_stress':listspeedpre,'Post_stress':listspeedpost,'S1':listspeed1,'S2':listspeed2,'S3':listspeed3,'S4':listspeed4,'S5':listspeed5,'S6':listspeed6,'S7':listspeed7,'S8':listspeed8,'S9':listspeed9,'S10':listspeed10,'S11':listspeed11,'S12':listspeed12,'S13':listspeed13,'S14':listspeed14,'S15':listspeed15,'S16':listspeed16,'S17':listspeed17,'S18':listspeed18, 'total':listtotalspeed})
speedtable.to_csv('data/Speed.csv')

# Turn Frequency

In [ ]:
for df1 in listdf1:
    listdirection=[]
    for i in range(1, len(df1.Recording_time)):
        if df1.X_center[i] > df1.X_center[i-1]:
            direction = 'right'
        elif df1.X_center[i] < df1.X_center[i-1]:
            direction ='left'
        listdirection.append(direction)
    listdirection.insert(0,0)
    df1['direction']=listdirection
    
    listturn=[]
    for i in range(3, len(df1.Recording_time)):
        if df1.direction[i] == df1.direction[i-1] and df1.direction[i-2] == df1.direction[i-3] and df1.direction[i-1]!= df1.direction[i-2]:
            turn=1
        else:
            turn=0
        listturn.append(turn)
    listturn.insert(0,0)
    listturn.insert(0,0)
    listturn.insert(0,0)
    df1['turns']=listturn
    
stress = speed_of_dot
prestress = Time_dots_start
poststress = length_of_trial-(Time_dots_start+(stims*stress))

listturnspost,listturnspre=[],[]
listturns1,listturns2,listturns3,listturns4,listturns5,listturns6,listturns7,listturns8,listturns9,listturns10,listturns11,listturns12=[],[],[],[],[],[],[],[],[],[],[],[]
listturns13,listturns14,listturns15,listturns16,listturns17,listturns18 = [],[],[],[],[],[]
for df1 in listdf1:
    summ=df1.groupby('phase')['turns'].sum()
    
    listturnspost.append((summ[0]/30))
    listturnspre.append((summ[1]/prestress))
    try:
        listturns1.append((summ[2]/stress))
    except:
        listturns1.append(0)
    try:
        listturns2.append((summ[3]/stress))
    except:
        listturns2.append(0)
    try:
        listturns3.append((summ[4]/stress))
    except:
        listturns3.append(0)
    try:
        listturns4.append((summ[5]/stress))
    except:
        listturns4.append(0)
    try:
        listturns5.append((summ[6]/stress))
    except:
        listturns5.append(0)
    try:
        listturns6.append((summ[7]/stress))
    except:
        listturns6.append(0)
    try:
        listturns7.append((summ[8]/stress))
    except:
        listturns7.append(0)
    try:
        listturns8.append((summ[9]/stress))
    except:
        listturns8.append(0)
    try:
        listturns9.append((summ[10]/stress))
    except:
        listturns9.append(0)
    try:
        listturns10.append((summ[11]/stress))
    except:
        listturns10.append(0)
    try:
        listturns11.append((summ[12]/stress))
    except:
        listturns11.append(0)
    try:
        listturns12.append((summ[13]/stress))
    except:
        listturns12.append(0)
    try:
        listturns13.append((summ[14]/stress))
    except:
        listturns13.append(0)
    try:
        listturns14.append((summ[15]/stress))
    except:
        listturns14.append(0)
    try:
        listturns15.append((summ[16]/stress))
    except:
        listturns15.append(0)
    try:
        listturns16.append((summ[17]/stress))
    except:
        listturns16.append(0)
    try:
        listturns17.append((summ[18]/stress))
    except:
        listturns17.append(0)
    try:
        listturns18.append((summ[19]/stress))
    except:
        listturns18.append(0)
    
turnstable = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'Pre_stress':listturnspre,'Post_stress':listturnspost,'S1':listturns1,'S2':listturns2,'S3':listturns3,'S4':listturns4,'S5':listturns5,'S6':listturns6,'S7':listturns7,'S8':listturns8,'S9':listturns9,'S10':listturns10,'S11':listturns11,'S12':listturns12,'S13':listturns13,'S14':listturns14,'S15':listturns15,'S16':listturns16,'S17':listturns17,'S18':listturns18})
turnstable.to_csv('data/Turns.csv')

In [ ]:
listturnspost,listturnspre=[],[]
listturns1,listturns2,listturns3,listturns4,listturns5,listturns6,listturns7,listturns8,listturns9,listturns10,listturns11,listturns12=[],[],[],[],[],[],[],[],[],[],[],[]
for df1 in listdf1:
    turnlength=0
    listturnlength=[]
    listtm=[]
    for i in range(0, len(df1.Recording_time)):
        if df1.turns[i] == 0:
            turnlength = turnlength + df1.Distance_moved[i]
            tm = df1.phase[i]
        else:
            listturnlength.append(turnlength)
            turnlength = 0
            tm = df1.phase[i]
            listtm.append(tm)
    
    
    turnlengthtable = pd.DataFrame({'phase':listtm,'turn_length':listturnlength})
                               
    summ=turnlengthtable.groupby('phase')['turn_length'].mean()
 
    listturnspost.append((summ[0]))
    listturnspre.append((summ[1]))
    try:
        listturns1.append((summ[2]))
    except:
        listturns1.append(0)
    try:
        listturns2.append((summ[3]))
    except:
        listturns2.append(0)
    try:
        listturns3.append((summ[4]))
    except:
        listturns3.append(0)
    try:
        listturns4.append((summ[5]))
    except:
        listturns4.append(0)
    try:
        listturns5.append((summ[6]))
    except:
        listturns5.append(0)
    try:
        listturns6.append((summ[7]))
    except:
        listturns6.append(0)
    try:
        listturns7.append((summ[8]))
    except:
        listturns7.append(0)
    try:
        listturns8.append((summ[9]))
    except:
        listturns8.append(0)
    try:
        listturns9.append((summ[10]))
    except:
        listturns9.append(0)
    try:
        listturns10.append((summ[11]))
    except:
        listturns10.append(0)
    try:
        listturns11.append((summ[12]))
    except:
        listturns11.append(0)
    try:
        listturns12.append((summ[13]))
    except:
        listturns12.append(0)
        
turnslengthtable = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'Pre_stress':listturnspre,'Post_stress':listturnspost,'S1':listturns1,'S2':listturns2,'S3':listturns3,'S4':listturns4,'S5':listturns5,'S6':listturns6,'S7':listturns7,'S8':listturns8,'S9':listturns9,'S10':listturns10,'S11':listturns11,'S12':listturns12})
turnslengthtable.to_csv('data/Turnslength.csv')

In [ ]:
pRint(inf)

In [ ]:
for df1 in listdf1:
    summ=df1.groupby('phase')['Distance_moved'].max()
    for i in range(0, len(df1.Recording_time)):
        if df1.phase[i]=='stimulus_1' and df1.Distance_moved[i] == summ[2]:
            print(df1.Recording_time[i]-Time_dots_start)

# Acceleration

In [ ]:
for df1 in listdf1:
    listacc=[]
    for i in range(1, len(df1.Recording_time)):
        acc = df1.Distance_moved[i]-df1.Distance_moved[i-1]
        listacc.append(acc)
    listacc.insert(0,0)
    df1['acceleration']=listacc
    
dfc = pd.concat(listdf1)
dfc.to_csv('data/all.csv');   

# Locomotion

In [ ]:
list0,list1,list2,list3,list4,list5 = [],[],[],[],[],[]
for df1 in listdf1:
    df1['bin'] = pd.cut(df['Recording_time'], [0, 5, 10,15,20,25,30,150])
    summ=df1.groupby('bin')['Distance_moved'].mean()
    list0.append(summ[0]*FPS)
    list1.append(summ[5]*FPS)
    list2.append(summ[10]*FPS)
    list3.append(summ[15]*FPS)
    list4.append(summ[20]*FPS)
    list5.append(summ[25]*FPS)
    
locomotion = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'bin0':list0, 'bin1':list1,'bin2':list2,'bin3':list3,'bin4':list4,'bin5':list5})
locomotion.to_csv('data/Locomotion.csv')

# Escape characteristics

In [ ]:
list0,list1,list2,list3,list4,list5,list6,list7,list8,list9 = [],[],[],[],[],[],[],[],[],[]
for df1 in listdf1:
    summ=df1.groupby('phase')['Distance_moved'].max()
    list0.append(summ[2]*FPS)
    list1.append(summ[3]*FPS)
    list2.append(summ[4]*FPS)
    list3.append(summ[5]*FPS)
    list4.append(summ[6]*FPS)
    list5.append(summ[7]*FPS)
    list6.append(summ[8]*FPS)
    try:
        list7.append(summ[9]*FPS)
        list8.append(summ[10]*FPS)
        list9.append(summ[11]*FPS)
    except:
        list7.append(0)
        list8.append(0)
        list9.append(0)
    
escape = pd.DataFrame({'Strain':listallstrain,'Treatment':listalltreatment,'max0':list0, 'max1':list1,'max2':list2,'max3':list3,'max4':list4,'max5':list5,'max6':list6,'max7':list7,'max8':list8,'max9':list9})
escape.to_csv('data/escape.csv')

# Print frame by frame for plots

In [ ]:
dot = '01'

dflip = pd.DataFrame()
n = 0
for df1 in listdf1:
    df=df1
    
    if df.strain[0] == 'StAR:bPAC-':
        df['Distance_moved'] = df['Distance_moved'] + 1
    elif df.strain[0] == 'TU':
        df['Distance_moved'] = df['Distance_moved'] + 2
    else:
        pass
        
        
    df = df[df.phase == 'stimulus_1'+str(dot)]
    df = df[df.treatment == 10]
    
    data = df.Distance_moved
    
    col_name = 'Subject' + str(n)
    dflip.loc[:, col_name] = data
    n += 1 
    
dflip.to_csv('data/dflip.csv')

In [ ]:
PRint(df1)

In [ ]:
#colors = ['#A658D5', '#2D7F32', '#5591C9', '#6D918A', '#3E0EA1', '#DA847A', '#2A93B3', '#0BF13F', '#7E352D', '#B211B8', '#460D87', '#7F8222', '#51D2B5']
FPSbegin = int(Time_dots_start*FPS)-FPS*5
FPSend = int(FPSbegin + ((10*speed_of_dot)*FPS))+FPS*15
Y = df1.Distance_moved[FPSbegin:FPSend]
X = df1.Recording_time[FPSbegin:FPSend]
plt.plot(X, Y);

In [ ]:
n = 1
for df1 in listdf1:
    phaselist = (df1['phase'].unique())
    colors = ['#A658D5', '#2D7F32', '#5591C9', '#6D918A', '#3E0EA1', '#DA847A', '#2A93B3', '#0BF13F', '#7E352D', '#B211B8', '#460D87', '#7F8222', '#51D2B5']
    plt.cla()
    for i in range(0, len(phaselist)):
        data = df1[df1['phase'] == phaselist[i]]
        Y = data.Distance_moved
        X = data.Recording_time
        plt.plot(X, Y, color = colors[i])
        plt.ylim([0, 2.5])
        plt.xlim([0, 150])
    plt.savefig('graphs/Trial_'+str(n))
    n=n+1

In [ ]:
from random import randint
selected_trial = 16

df1 = listdf1[selected_trial]
phaselist = (df1['phase'].unique())

colors = ['#A658D5', '#2D7F32', '#5591C9', '#6D918A', '#3E0EA1', '#DA847A', '#2A93B3', '#0BF13F', '#7E352D', '#B211B8', '#460D87', '#7F8222', '#51D2B5']

#colors=[]
#for i in range(0, len(phaselist)):
#    colors.append('#%06X' % randint(0, 0xFFFFFF))

for i in range(0, len(phaselist)):
    data = df1[df1['phase'] == phaselist[i]]
    Y = data.Distance_moved
    X = data.Recording_time
    plt.plot(X, Y, color = colors[i])
    plt.ylim([0, 2.5])